In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wrangle import note, low_occurance
import category_encoders as ce
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score


pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
data         = pd.read_csv('train_features.csv')
feature_info = pd.read_csv('train_labels.csv')

feature_info = feature_info.drop(columns = 'id')

data = low_occurance(note(pd.concat([data, feature_info], 
                                    sort = False, axis = 1)))

data.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,small_fry,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,small_fry,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [5]:
def finish_dataframe (data=data):
    
    df = data[['latitude','longitude','gps_height','population','ward',
               'construction_year','amount_tsh','quantity','funder',
               'waterpoint_type', 'status_group','extraction_type',
               'source','management']]

    df['longitude'] = df['longitude'].replace(0 ,np.nan)
    df['latitude'] = df['latitude'].replace(-2.000000e-08 ,np.nan)

    df = df.dropna()
    

    n_clusters = [50, 6000]

    for n in n_clusters:
        kmeans = KMeans(n_clusters=n, n_jobs=-1)
        kmeans.fit(df[['longitude','latitude']])
        y_kmeans = kmeans.predict(df[['longitude','latitude']])

        df[str(n)+'_kmeans_clusters'] = y_kmeans

    df = df.drop(columns=['latitude','longitude'])
    
    ordinal = ce.OrdinalEncoder(return_df=True)
    df = ordinal.fit_transform(df)
    
    return df

df = finish_dataframe()

In [35]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', max_iter=10000, multi_class='auto')

In [51]:
from sklearn.model_selection import cross_val_score

X = df.drop(columns='status_group')
y = df['status_group']

model.fit(X , y)

score  = (cross_val_score(model, X, y, 
                          cv = 10, 
                          scoring = 'accuracy',
                          n_jobs = -1,
                          verbose = 10))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:  2.5min remaining:  5.9min
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  2.8min remaining:  2.8min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  3.1min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  3.9min finished


In [52]:
print('Cross Validation Spread: ', score)
print('Cross Validation Mean: ', score.mean())
print('Cross Validation STDV: ', np.std(score))

Cross Validation Spread:  [0.63579103 0.63764357 0.63208596 0.63016491 0.63572355 0.6386212
 0.63843588 0.62861379 0.64003707 0.63410565]
Cross Validation Mean:  0.6351222611775732
Cross Validation STDV:  0.0036294907152368524


In [7]:
from xgboost import XGBClassifier

xgbc = XGBClassifier(booster='gbtree', colsample_bylevel=1, colsample_bytree=1, max_delta_step=0, 
                     max_depth=17, min_child_weight=1, missing=None, n_estimators=100, n_jobs=-1, 
                     objective='multi:softmax',num_class=3,andom_state=42, reg_alpha=0, reg_lambda=1, 
                     scale_pos_weight=1,seed=42, silent=True, subsample=1, eval_metric='merror')

In [56]:
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'base_score': np.arange(start=0.3, stop=0.72, step=0.02),
    'max_depth': np.arange(start=1, stop=100),
    'n_estimators': np.arange(start=50, stop=500),
}

RSCV = RandomizedSearchCV(estimator=xgbc, 
                          param_distributions=param_distributions, 
                          n_iter=120, 
                          scoring='accuracy', 
                          n_jobs=-1, 
                          cv=5, 
                          verbose=10, 
                          random_state=42, 
                          return_train_score=True)

X = df.drop(columns='status_group')
y = df['status_group']

RSCV.fit(X, y)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 25.1min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 29.2min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed: 33.5min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 49.6min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed: 58.6min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 65.4min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 79.8min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 87.1min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 108.4min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 129.0min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed: 146.2min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=XGBClassifier(andom_state=42, base_score=0.5,
                                           booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1,
                                           eval_metric='merror', gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=17, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=-1, nthread=None, num_class=3,
                                           obje...
       427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439,
       440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452,
       453, 454, 455, 456, 457, 45

In [58]:
results_RSCV = pd.DataFrame(RSCV.cv_results_)
results_RSCV.sort_values(by='rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,param_base_score,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
86,179.836953,4.776229,7.239346,0.837258,439,9,0.66,"{'n_estimators': 439, 'max_depth': 9, 'base_sc...",0.810080,0.807671,...,0.808194,0.003300,1,0.939868,0.944107,0.942602,0.944504,0.946148,0.943446,0.002115
22,108.149912,4.663385,3.708100,0.419618,165,13,0.62,"{'n_estimators': 165, 'max_depth': 13, 'base_s...",0.809802,0.802853,...,0.805878,0.002919,2,0.958839,0.960067,0.961318,0.959374,0.962454,0.960410,0.001316
80,99.125538,5.093529,3.532050,0.588358,142,14,0.7,"{'n_estimators': 142, 'max_depth': 14, 'base_s...",0.809802,0.802390,...,0.805433,0.003151,3,0.964861,0.963935,0.965116,0.963705,0.966345,0.964793,0.000942
93,46.376212,0.366455,1.080350,0.089693,86,11,0.42,"{'n_estimators': 86, 'max_depth': 11, 'base_sc...",0.807022,0.801927,...,0.804951,0.002914,4,0.904823,0.907787,0.908204,0.905846,0.911220,0.907576,0.002204
9,55.441087,1.074257,0.971262,0.046881,137,9,0.36,"{'n_estimators': 137, 'max_depth': 9, 'base_sc...",0.804614,0.803224,...,0.803654,0.003335,5,0.882401,0.886639,0.883906,0.882035,0.888938,0.884784,0.002634


In [61]:
RSCV.best_estimator_

XGBClassifier(andom_state=42, base_score=0.6600000000000004, booster='gbtree',
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              eval_metric='merror', gamma=0, learning_rate=0.1,
              max_delta_step=0, max_depth=9, min_child_weight=1, missing=None,
              n_estimators=439, n_jobs=-1, nthread=None, num_class=3,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=42, silent=True,
              subsample=1, verbosity=1)

In [65]:
from sklearn.model_selection import GridSearchCV

param_distributions = {
    'base_score': [0.65],
    'max_depth': [9,10,11,12,13,14],
    'gamma': [0,1,5],
    'n_estimators': [137, 142, 165]
}




GSCV = GridSearchCV(estimator=xgbc, 
                    param_grid=param_distributions, 
                    scoring='accuracy', 
                    n_jobs=-1,
                    refit=True, 
                    cv=5, 
                    verbose=10, 
                    return_train_score=True)

X = df.drop(columns='status_group')
y = df['status_group']

GSCV.fit(X, y)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed: 14.0min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed: 17.5min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed: 19.8min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 22.0min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed: 24.8min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 27.8min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed: 32.1min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 35

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBClassifier(andom_state=42, base_score=0.5,
                                     booster='gbtree', colsample_bylevel=1,
                                     colsample_bynode=1, colsample_bytree=1,
                                     eval_metric='merror', gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=17, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=-1,
                                     nthread=None, num_class=3,
                                     objective='multi:softmax', random_state=0,
                                     reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=42, silent=True,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=-1,
             param_grid={'base_score': [0.65],

In [67]:
results_GSCV = pd.DataFrame(GSCV.cv_results_)
results_GSCV.sort_values(by='rank_test_score').head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_base_score,param_gamma,param_max_depth,param_n_estimators,params,split0_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
24,68.512402,0.241346,1.460493,0.216840,0.65,1,11,137,"{'base_score': 0.65, 'gamma': 1, 'max_depth': ...",0.812859,...,0.808250,0.003376,1,0.902553,0.902900,0.902205,0.900380,0.905916,0.902791,0.001788
25,70.583746,0.380141,1.429504,0.121961,0.65,1,11,142,"{'base_score': 0.65, 'gamma': 1, 'max_depth': ...",0.812581,...,0.808138,0.003369,2,0.903085,0.903062,0.902761,0.900959,0.906541,0.903282,0.001810
26,80.888325,0.513623,1.650179,0.293211,0.65,1,11,165,"{'base_score': 0.65, 'gamma': 1, 'max_depth': ...",0.811469,...,0.807935,0.002428,3,0.906861,0.905425,0.906652,0.903599,0.909112,0.906330,0.001811
10,77.845087,1.459640,2.927861,0.560442,0.65,0,12,142,"{'base_score': 0.65, 'gamma': 0, 'max_depth': ...",0.811562,...,0.807768,0.003162,4,0.943227,0.944154,0.945543,0.942952,0.945013,0.944178,0.000997
9,77.031683,2.508786,2.601822,0.668278,0.65,0,12,137,"{'base_score': 0.65, 'gamma': 0, 'max_depth': ...",0.810636,...,0.807749,0.003081,5,0.941837,0.942231,0.944200,0.941632,0.943322,0.942644,0.000972


In [70]:
np.arange(start=0, stop=5, step=0.2)

array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. , 2.2, 2.4,
       2.6, 2.8, 3. , 3.2, 3.4, 3.6, 3.8, 4. , 4.2, 4.4, 4.6, 4.8])

In [44]:
from skopt import BayesSearchCV

X = df.drop(columns='status_group')
y = df['status_group']

param_grid = {
    'base_score': (0.3, 0.7),
    'max_depth': (1, 100),
    'n_estimators': (50, 500),
    'gamma': (0, 5),
    'learning_rate': (0.01, 0.3)
}

BSCV = BayesSearchCV (estimator=xgbc, 
                      search_spaces=param_grid, 
                      optimizer_kwargs=None,
                      n_iter=40, 
                      scoring='accuracy', 
                      fit_params=None, 
                      n_jobs=-1,
                      n_points=10,
                      cv=3, 
                      verbose=10,
                      random_state=42,
                      return_train_score=True)
    
BSCV.fit(X, y)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  7.0min
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed: 16.0min remaining:  9.3min
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed: 19.5min remaining:  5.9min
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed: 22.4min remaining:  2.5min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 28.4min finished


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   52.8s
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:  1.6min remaining:   56.6s
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:  2.8min remaining:   51.5s
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:  3.9min remaining:   25.8s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  5.8min finished


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 10.0min
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed: 23.3min remaining: 13.5min
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed: 27.0min remaining:  8.2min
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed: 29.1min remaining:  3.2min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 29.4min finished


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed: 15.4min remaining:  8.9min
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed: 20.4min remaining:  6.2min
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed: 21.2min remaining:  2.4min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 22.3min finished


BayesSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(andom_state=42, base_score=0.5, booster='gbtree',
       colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
       eval_metric='merror', gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=17, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, num_class=3, objective='multi:softmax',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=42, silent=True, subsample=1, verbosity=1),
       fit_params=None, iid=True, n_iter=40, n_jobs=-1, n_points=10,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=42,
       refit=True, return_train_score=True, scoring='accuracy',
       search_spaces={'base_score': (0.3, 0.7), 'max_depth': (1, 100), 'n_estimators': (50, 500), 'gamma': (0, 5), 'learning_rate': (0.01, 0.3)},
       verbose=10)

In [37]:
derp = pd.DataFrame(BSCV.cv_results_).sort_values(by='rank_test_score')

In [43]:
derp

,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,...,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_base_score,param_gamma,param_learning_rate,param_max_depth,param_n_estimators,params
1,0.799044,0.804770,0.799533,0.801116,0.002592,1,0.934236,0.935236,0.934793,0.934755,...,339.056964,1.817613,20.713698,1.434731,0.465647,1,0.01,31,341,"{'base_score': 0.4656474529942154, 'gamma': 1,..."
10,0.800767,0.804603,0.799144,0.801505,0.002289,1,0.929372,0.930150,0.930624,0.930049,...,301.375444,0.488189,18.720457,0.258600,0.300000,1,0.01,19,373,"{'base_score': 0.3, 'gamma': 1, 'learning_rate..."
5,0.799544,0.803713,0.798088,0.800448,0.002384,2,0.931206,0.932040,0.931152,0.931466,...,285.210791,3.362891,14.540409,0.590180,0.582179,1,0.01,49,289,"{'base_score': 0.5821792788079154, 'gamma': 1,..."
12,0.800656,0.804269,0.797865,0.800930,0.002622,2,0.926342,0.927009,0.927317,0.926889,...,250.886274,4.732437,14.010658,0.419361,0.300000,1,0.01,19,308,"{'base_score': 0.3, 'gamma': 1, 'learning_rate..."
0,0.797487,0.800823,0.797865,0.798725,0.001491,3,0.860022,0.859799,0.862388,0.860736,...,355.507468,5.443322,21.275508,1.721243,0.464042,3,0.01,32,351,"{'base_score': 0.4640415835413255, 'gamma': 3,..."
16,0.799655,0.802824,0.797198,0.799893,0.002303,3,0.920477,0.921728,0.921535,0.921247,...,200.979956,0.478130,12.037283,1.745370,0.300000,1,0.01,19,227,"{'base_score': 0.3, 'gamma': 1, 'learning_rate..."
15,0.799655,0.802824,0.797198,0.799893,0.002303,3,0.920477,0.921728,0.921535,0.921247,...,216.767523,9.466301,14.068042,2.567897,0.300000,1,0.01,19,227,"{'base_score': 0.3, 'gamma': 1, 'learning_rate..."
14,0.799655,0.802824,0.797198,0.799893,0.002303,3,0.920477,0.921728,0.921535,0.921247,...,201.897935,4.391327,16.506685,2.034995,0.300000,1,0.01,19,227,"{'base_score': 0.3, 'gamma': 1, 'learning_rate..."
13,0.799655,0.802824,0.797198,0.799893,0.002303,3,0.920477,0.921728,0.921535,0.921247,...,187.371574,5.406776,13.491356,0.280905,0.300000,1,0.01,19,227,"{'base_score': 0.3, 'gamma': 1, 'learning_rate..."
11,0.799655,0.802824,0.797198,0.799893,0.002303,3,0.920477,0.921728,0.921535,0.921247,...,180.827302,0.332009,11.423305,0.114447,0.300000,1,0.01,19,227,"{'base_score': 0.3, 'gamma': 1, 'learning_rate..."
